In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

from skforecast.exceptions import OneStepAheadValidationWarning
from skforecast.datasets import fetch_dataset
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate

In [24]:
y = pd.Series(np.arange(20), name='y')

In [36]:
forecaster = ForecasterRecursive(
                 regressor = LinearRegression(),
                 lags      = 5,
                 differentiation = 2
             )
forecaster.differentiator.window_size = forecaster.window_size
forecaster.fit(y=y)
forecaster.differentiator.window_size

7

In [37]:
y

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
Name: y, dtype: int32

In [38]:
# Create training matrices
# ==============================================================================
X_train, y_train = forecaster.create_train_X_y(y=y)
y_train.head(3)

7    0.0
8    0.0
9    0.0
Name: y, dtype: float64

In [39]:
forecaster.differentiator.pre_train_values

[5, 1]

In [40]:
X = forecaster.regressor.predict(X_train)
X

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [41]:
X_undiff = np.insert(X, 0, 1)
X_undiff = np.cumsum(X_undiff, dtype=float)
X_undiff = np.insert(X_undiff, 0, 5)
X_undiff = np.cumsum(X_undiff, dtype=float)
X_undiff

array([ 5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17.,
       18., 19.])

In [42]:
y[5 + 2:].to_numpy()

array([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [43]:
forecaster.differentiator.inverse_transform_training(X)

array([ 7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19.])

In [46]:
pre_train_values = []

if not pre_train_values:
    print('hola')

hola


In [47]:
forecaster = ForecasterDirect(
                 regressor = LinearRegression(),
                 steps = 2,
                 lags      = 5,
                 differentiation = None
             )
# forecaster.differentiator.window_size = forecaster.window_size
forecaster.fit(y=y)
# forecaster.differentiator.window_size

In [48]:
# Create the whole train matrix
X_train, y_train = forecaster.create_train_X_y(y=y)

# Extract X and y to train the model for step 1
X_train_1, y_train_1 = forecaster.filter_train_X_y_for_step(
                           step          = 2,
                           X_train       = X_train,
                           y_train       = y_train,
                           remove_suffix = False
                       )

y_train_1.head(4)

6    6.0
7    7.0
8    8.0
9    9.0
Name: y_step_2, dtype: float64

In [22]:
X = forecaster.regressors_[2].predict(X_train)
forecaster.differentiator.inverse_transform_training(X)

AttributeError: 'NoneType' object has no attribute 'inverse_transform_training'

In [13]:
from skforecast.preprocessing import TimeSeriesDifferentiator

y = np.array([1, 4, 8, 10, 13, 22, 40, 46], dtype=float)
y_diff_1 = np.array([np.nan, 3., 4., 2., 3., 9., 18., 6.])
y_diff_2 = np.array([np.nan, np.nan, 1., -2., 1., 6., 9., -12.])
y_diff_3 = np.array([np.nan, np.nan, np.nan, -3., 3., 5., 3., -21.])

In [14]:
order = 1
y_diff = y_diff_1

window_size = 3  # 2 LAGS + 1 diff

y_train = y[window_size:]
y_diff_train = y_diff[window_size:]
assert len(y_train) == len(y_diff_train)

transformer = TimeSeriesDifferentiator(order=order, window_size=window_size)
transformer.fit_transform(y)
print(transformer.pre_train_values)
print(y_diff_train)
results = transformer.inverse_transform_training(y_diff_train)

np.testing.assert_array_equal(results, y_train)

[8.0]
[ 2.  3.  9. 18.  6.]
